In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
data = pd.read_csv('Aggregated User Data.csv')
data = data[(data['UUID'] == '0BFC35E2-4817-4865-BFA7-764742302A2D') | (data['UUID'] == '0A986513-7828-4D53-AA1F-E02D6DF9561B') | (data['UUID'] == '00EABED2-271D-49D8-B599-1D4A09240601')] 
data.drop(columns = ['timestamp'], inplace = True)

In [3]:
def interpolation(df):
    col_to_avg = list(df.columns) #Start with keeping all the columns as columns to use an average interpolation on
    for k in range(len(list(df.columns))):
        if list(df.columns)[k].startswith(('discrete', 'label')): #Remove label and discrete columns from col_to_avg
            col_to_avg.remove(list(df.columns)[k])
    
    df_with_avg = df[col_to_avg].fillna(df[col_to_avg].mean()) #Interpolate nan columns for all continuous-valued columns with average
    
    col_to_zero = list(df.columns)
    for k in range(len(list(df.columns))):
        if not list(df.columns)[k].startswith(('discrete', 'label')): #Remove all columns except label and discrete
            col_to_zero.remove(list(df.columns)[k])
    
    df_with_zero = df[col_to_zero].fillna(0) #Interpolate nan values for label and discrete columns with 0
    
    return pd.concat([df_with_avg, df_with_zero], axis = 1)

In [4]:
data.iloc[:,1:27] = interpolation(data.iloc[:,1:27])
data = data.iloc[:,:27].values
data

array([['00EABED2-271D-49D8-B599-1D4A09240601', 0.996815, 0.003529, ...,
        0.10692, 0.516842, 0.255494],
       ['00EABED2-271D-49D8-B599-1D4A09240601', 0.996864, 0.004172, ...,
        -0.079483, 0.357748, 0.036252],
       ['00EABED2-271D-49D8-B599-1D4A09240601', 0.996825, 0.003667, ...,
        0.122432, 0.464881, 0.222375],
       ...,
       ['0BFC35E2-4817-4865-BFA7-764742302A2D', 1.012776, 0.008135, ...,
        -0.099042, -0.307551, -0.592356],
       ['0BFC35E2-4817-4865-BFA7-764742302A2D', 1.013261, 0.013788, ...,
        0.03825, -0.251999, -0.714921],
       ['0BFC35E2-4817-4865-BFA7-764742302A2D', 1.004676,
        0.016819999999999998, ..., 0.566422, -0.23814, -0.670271]],
      dtype=object)

In [8]:
X = data[:, 1:]
y = data[:, 0]

y = LabelEncoder().fit_transform(y)

In [12]:
# ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [0])], remainder = 'passthrough')
# data = np.array(ct.fit_transform(data))
# data

In [13]:
# X = data[:,3:]
# y = data[:,:3]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[-0.27611387,  0.21757044,  1.13548203, ...,  0.69508287,
        -0.4076953 , -1.47963416],
       [ 0.64180031,  1.50223851,  2.09574985, ..., -2.26289883,
        -1.92735886,  2.34861968],
       [-0.30106694, -0.40921879, -0.35798907, ..., -0.74425   ,
         0.7087755 ,  1.10802921],
       ...,
       [ 0.51940534, -0.39594064, -0.35183917, ...,  0.73333413,
         2.51988294,  1.30078982],
       [ 0.61098535, -0.13112532, -0.21546569, ...,  1.78638596,
         2.50193401,  1.8651583 ],
       [-0.45011268, -0.39209821, -0.34295903, ...,  1.31044205,
        -0.28391418,  0.67770612]])

In [16]:
# y_train = y_train.astype(int)
# y_test = y_test.astype(int)

In [17]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = 26):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 50),
            classifier_block(50, 100),
            classifier_block(100, 75),
            nn.Linear(75, 3) 
        )
    def forward(self, x):
        return self.network(x)

In [18]:
model = Classifier().to(device)
lr = 0.01
n_epochs = 2000
batch_size = 1000

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train).to(device)
train_labels = torch.tensor(y_train).to(device)
test_features = torch.tensor(X_test).to(device)
test_labels = torch.tensor(y_test).to(device)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [21]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        #_, encoded_labels = torch.max(labels, dim = 1) 
        
        optimizer.zero_grad()
        preds = model(features.float())
        loss = criterion(preds, labels.long())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 6.735054194927216, Final Batch Loss: 0.6211680769920349
Epoch 2, Loss: 4.22953674197197, Final Batch Loss: 0.41681239008903503
Epoch 3, Loss: 3.4087719917297363, Final Batch Loss: 0.44122084975242615
Epoch 4, Loss: 2.8608688414096832, Final Batch Loss: 0.3930571973323822
Epoch 5, Loss: 2.432162821292877, Final Batch Loss: 0.3062441945075989
Epoch 6, Loss: 2.1739612370729446, Final Batch Loss: 0.24381034076213837
Epoch 7, Loss: 2.0033541172742844, Final Batch Loss: 0.23432736098766327
Epoch 8, Loss: 1.7365903109312057, Final Batch Loss: 0.18334990739822388
Epoch 9, Loss: 1.6655087620019913, Final Batch Loss: 0.2612376809120178
Epoch 10, Loss: 1.5306715667247772, Final Batch Loss: 0.1643196940422058
Epoch 11, Loss: 1.435188814997673, Final Batch Loss: 0.15852637588977814
Epoch 12, Loss: 1.4262256175279617, Final Batch Loss: 0.17621558904647827
Epoch 13, Loss: 1.3879422396421432, Final Batch Loss: 0.16894973814487457
Epoch 14, Loss: 1.2539493292570114, Final Batch Loss: 0.1

Epoch 113, Loss: 0.5045953467488289, Final Batch Loss: 0.06141742691397667
Epoch 114, Loss: 0.5109622962772846, Final Batch Loss: 0.05924179032444954
Epoch 115, Loss: 0.49291060492396355, Final Batch Loss: 0.060892295092344284
Epoch 116, Loss: 0.4501713253557682, Final Batch Loss: 0.04159226268529892
Epoch 117, Loss: 0.48676639795303345, Final Batch Loss: 0.06700034439563751
Epoch 118, Loss: 0.44569724798202515, Final Batch Loss: 0.04180705547332764
Epoch 119, Loss: 0.5026847459375858, Final Batch Loss: 0.07488467544317245
Epoch 120, Loss: 0.4699641317129135, Final Batch Loss: 0.03645701706409454
Epoch 121, Loss: 0.5892842337489128, Final Batch Loss: 0.06682207435369492
Epoch 122, Loss: 0.6724583581089973, Final Batch Loss: 0.07353845983743668
Epoch 123, Loss: 0.6146304421126842, Final Batch Loss: 0.0653533786535263
Epoch 124, Loss: 0.5218975469470024, Final Batch Loss: 0.04150962457060814
Epoch 125, Loss: 0.5937473550438881, Final Batch Loss: 0.06791853159666061
Epoch 126, Loss: 0.507

Epoch 224, Loss: 0.3957103583961725, Final Batch Loss: 0.07128074765205383
Epoch 225, Loss: 0.390963152050972, Final Batch Loss: 0.052848488092422485
Epoch 226, Loss: 0.48195549845695496, Final Batch Loss: 0.06755262613296509
Epoch 227, Loss: 0.4733837805688381, Final Batch Loss: 0.059654075652360916
Epoch 228, Loss: 0.4037296697497368, Final Batch Loss: 0.040469687432050705
Epoch 229, Loss: 0.44384387508034706, Final Batch Loss: 0.05693346634507179
Epoch 230, Loss: 0.44253087788820267, Final Batch Loss: 0.07551164925098419
Epoch 231, Loss: 0.3850848749279976, Final Batch Loss: 0.04315156489610672
Epoch 232, Loss: 0.37432247027754784, Final Batch Loss: 0.03611272946000099
Epoch 233, Loss: 0.42125309258699417, Final Batch Loss: 0.08660880476236343
Epoch 234, Loss: 0.4176378399133682, Final Batch Loss: 0.047082725912332535
Epoch 235, Loss: 0.4087029341608286, Final Batch Loss: 0.029148729518055916
Epoch 236, Loss: 0.4284442439675331, Final Batch Loss: 0.05866112932562828
Epoch 237, Loss:

Epoch 334, Loss: 0.30187773145735264, Final Batch Loss: 0.02295215241611004
Epoch 335, Loss: 0.38967857509851456, Final Batch Loss: 0.04587370902299881
Epoch 336, Loss: 0.39148327708244324, Final Batch Loss: 0.04550326243042946
Epoch 337, Loss: 0.30409443750977516, Final Batch Loss: 0.028710046783089638
Epoch 338, Loss: 0.30803424678742886, Final Batch Loss: 0.02061139978468418
Epoch 339, Loss: 0.34132677875459194, Final Batch Loss: 0.05113082751631737
Epoch 340, Loss: 0.35760893672704697, Final Batch Loss: 0.050360992550849915
Epoch 341, Loss: 0.41078997030854225, Final Batch Loss: 0.08307570964097977
Epoch 342, Loss: 0.3773187045007944, Final Batch Loss: 0.024948669597506523
Epoch 343, Loss: 0.3477020673453808, Final Batch Loss: 0.041925933212041855
Epoch 344, Loss: 0.3705424889922142, Final Batch Loss: 0.04839404672384262
Epoch 345, Loss: 0.4515247493982315, Final Batch Loss: 0.056701455265283585
Epoch 346, Loss: 0.3645300194621086, Final Batch Loss: 0.032755278050899506
Epoch 347, 

Epoch 444, Loss: 0.3095229882746935, Final Batch Loss: 0.042260412126779556
Epoch 445, Loss: 0.30979782715439796, Final Batch Loss: 0.037966955453157425
Epoch 446, Loss: 0.3553644735366106, Final Batch Loss: 0.04706062376499176
Epoch 447, Loss: 0.3057486657053232, Final Batch Loss: 0.04515565186738968
Epoch 448, Loss: 0.320608526468277, Final Batch Loss: 0.04154535382986069
Epoch 449, Loss: 0.35332902520895004, Final Batch Loss: 0.033810053020715714
Epoch 450, Loss: 0.33058640360832214, Final Batch Loss: 0.03332697972655296
Epoch 451, Loss: 0.333791920915246, Final Batch Loss: 0.04738568887114525
Epoch 452, Loss: 0.310412235558033, Final Batch Loss: 0.015990914776921272
Epoch 453, Loss: 0.3077931944280863, Final Batch Loss: 0.04060530290007591
Epoch 454, Loss: 0.29766105115413666, Final Batch Loss: 0.04616115987300873
Epoch 455, Loss: 0.2793842926621437, Final Batch Loss: 0.02594231627881527
Epoch 456, Loss: 0.2983043994754553, Final Batch Loss: 0.050118301063776016
Epoch 457, Loss: 0.

...

Epoch 1425, Loss: 0.29557582177221775, Final Batch Loss: 0.04203776642680168
Epoch 1426, Loss: 0.2834184616804123, Final Batch Loss: 0.07385009527206421
Epoch 1427, Loss: 0.2827990176156163, Final Batch Loss: 0.00910127256065607
Epoch 1428, Loss: 0.2536627370864153, Final Batch Loss: 0.03525436669588089
Epoch 1429, Loss: 0.24881750252097845, Final Batch Loss: 0.035074807703495026
Epoch 1430, Loss: 0.2446755226701498, Final Batch Loss: 0.051871102303266525
Epoch 1431, Loss: 0.20375591423362494, Final Batch Loss: 0.012992972508072853
Epoch 1432, Loss: 0.23090241197496653, Final Batch Loss: 0.03273588418960571
Epoch 1433, Loss: 0.21969191916286945, Final Batch Loss: 0.028569955378770828
Epoch 1434, Loss: 0.25061461329460144, Final Batch Loss: 0.030976198613643646
Epoch 1435, Loss: 0.2442425861954689, Final Batch Loss: 0.04947769641876221
Epoch 1436, Loss: 0.28257505130022764, Final Batch Loss: 0.014178820885717869
Epoch 1437, Loss: 0.47791279666125774, Final Batch Loss: 0.1509659141302108

Epoch 1638, Loss: 0.21322588808834553, Final Batch Loss: 0.02972034178674221
Epoch 1639, Loss: 0.20479553565382957, Final Batch Loss: 0.007846519351005554
Epoch 1640, Loss: 0.20265874825417995, Final Batch Loss: 0.03597131744027138
Epoch 1641, Loss: 0.18512485921382904, Final Batch Loss: 0.011937817558646202
Epoch 1642, Loss: 0.192008838057518, Final Batch Loss: 0.023199614137411118
Epoch 1643, Loss: 0.1848226971924305, Final Batch Loss: 0.018009459599852562
Epoch 1644, Loss: 0.24419464357197285, Final Batch Loss: 0.02734561078250408
Epoch 1645, Loss: 0.1947005558758974, Final Batch Loss: 0.022829756140708923
Epoch 1646, Loss: 0.22837405651807785, Final Batch Loss: 0.01222585141658783
Epoch 1647, Loss: 0.2500706799328327, Final Batch Loss: 0.022446200251579285
Epoch 1648, Loss: 0.25243395287543535, Final Batch Loss: 0.019621625542640686
Epoch 1649, Loss: 0.19089177530258894, Final Batch Loss: 0.027525821700692177
Epoch 1650, Loss: 0.18734107166528702, Final Batch Loss: 0.03795715048909

Epoch 1745, Loss: 0.20991212129592896, Final Batch Loss: 0.0324358195066452
Epoch 1746, Loss: 0.21050655748695135, Final Batch Loss: 0.02126932330429554
Epoch 1747, Loss: 0.17953157611191273, Final Batch Loss: 0.013146908953785896
Epoch 1748, Loss: 0.22694813180714846, Final Batch Loss: 0.008879194967448711
Epoch 1749, Loss: 0.18784539308398962, Final Batch Loss: 0.012972264550626278
Epoch 1750, Loss: 0.19558235257863998, Final Batch Loss: 0.026351148262619972
Epoch 1751, Loss: 0.20960234478116035, Final Batch Loss: 0.018770437687635422
Epoch 1752, Loss: 0.2763970382511616, Final Batch Loss: 0.03578527271747589
Epoch 1753, Loss: 0.44884019531309605, Final Batch Loss: 0.024343932047486305
Epoch 1754, Loss: 0.3425292447209358, Final Batch Loss: 0.028286686167120934
Epoch 1755, Loss: 0.31964097917079926, Final Batch Loss: 0.0506352037191391
Epoch 1756, Loss: 0.3226358015090227, Final Batch Loss: 0.05440318211913109
Epoch 1757, Loss: 0.24202887248247862, Final Batch Loss: 0.010998375713825

Epoch 1852, Loss: 0.23282254114747047, Final Batch Loss: 0.009879661723971367
Epoch 1853, Loss: 0.22729207016527653, Final Batch Loss: 0.027870016172528267
Epoch 1854, Loss: 0.193122205324471, Final Batch Loss: 0.027207380160689354
Epoch 1855, Loss: 0.21105147246271372, Final Batch Loss: 0.015211612917482853
Epoch 1856, Loss: 0.17602716200053692, Final Batch Loss: 0.02259019948542118
Epoch 1857, Loss: 0.20403220131993294, Final Batch Loss: 0.02769141085445881
Epoch 1858, Loss: 0.21132636070251465, Final Batch Loss: 0.01851705089211464
Epoch 1859, Loss: 0.23104576114565134, Final Batch Loss: 0.009857124648988247
Epoch 1860, Loss: 0.21348951943218708, Final Batch Loss: 0.016767974942922592
Epoch 1861, Loss: 0.1882298979908228, Final Batch Loss: 0.03057495877146721
Epoch 1862, Loss: 0.25698004849255085, Final Batch Loss: 0.013300368562340736
Epoch 1863, Loss: 0.26122440584003925, Final Batch Loss: 0.030748531222343445
Epoch 1864, Loss: 0.16910607554018497, Final Batch Loss: 0.020937541499

Epoch 1959, Loss: 0.22490816190838814, Final Batch Loss: 0.03356257081031799
Epoch 1960, Loss: 0.22307249996811152, Final Batch Loss: 0.011985612101852894
Epoch 1961, Loss: 0.24628400336951017, Final Batch Loss: 0.01549364160746336
Epoch 1962, Loss: 0.3530259355902672, Final Batch Loss: 0.017593001946806908
Epoch 1963, Loss: 0.3493844736367464, Final Batch Loss: 0.03802330791950226
Epoch 1964, Loss: 0.2592899929732084, Final Batch Loss: 0.04576262831687927
Epoch 1965, Loss: 0.2916895840317011, Final Batch Loss: 0.05532991141080856
Epoch 1966, Loss: 0.24068351089954376, Final Batch Loss: 0.02164650149643421
Epoch 1967, Loss: 0.2735073585063219, Final Batch Loss: 0.018932469189167023
Epoch 1968, Loss: 0.2559840064495802, Final Batch Loss: 0.03764191269874573
Epoch 1969, Loss: 0.2105716923251748, Final Batch Loss: 0.011794275604188442
Epoch 1970, Loss: 0.20613751839846373, Final Batch Loss: 0.02697768062353134
Epoch 1971, Loss: 0.21771325916051865, Final Batch Loss: 0.03848421946167946
Ep

In [22]:
softmax = nn.Softmax(dim = 1)
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    #_, answers = torch.max(labels, dim = 1)
    print(metrics.confusion_matrix(labels, preds))
    print(metrics.classification_report(labels, preds, digits = 3))

[[401  37   2]
 [ 22 775   6]
 [  6   2 620]]
              precision    recall  f1-score   support

           0      0.935     0.911     0.923       440
           1      0.952     0.965     0.959       803
           2      0.987     0.987     0.987       628

    accuracy                          0.960      1871
   macro avg      0.958     0.955     0.956      1871
weighted avg      0.960     0.960     0.960      1871

